In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
from matplotlib.ticker import MaxNLocator
import matplotlib.dates as mdates
import os

## Read in data from loops

directory_path = r'C:\Users\mrpro\Documents\Code\CBI\Corrected_Data_Official\nesscanResult_removedBadNesdisRecords\PortIsabel'
dataframes = []


for filename in sorted(os.listdir(directory_path)):
    if filename.endswith('.csv'):

        file_path = os.path.join(directory_path, filename)
        
        df = pd.read_csv(
        file_path,
        parse_dates=[0],
        na_values=[-999, -99, 99, 'NA', 'RM'],
        engine='python'   
        )
        df.columns = ['station #','date', 'pwl',]
        dataframes.append(df)

pi = pd.concat(dataframes, ignore_index=True)

directory_path = r'C:\Users\mrpro\Documents\Code\CBI\data 2\NOAA\portIsabel'
dataframes = []

# Loop through the files in the directory
for filename in sorted(os.listdir(directory_path)):
    if filename.endswith('level.csv'):

        file_path = os.path.join(directory_path, filename)
        
        
        df = pd.read_csv(file_path)
        
        dataframes.append(df)

#formats the dataframes
pi_NOAA = pd.concat(dataframes, ignore_index=True)
pi_NOAA.drop(labels=range(0,3), axis = 0,inplace=True)
pi_NOAA.reset_index(drop = True, inplace=True)
pi['date'] = pd.to_datetime(pi['date'])
pi_NOAA['date'] = pd.to_datetime(pi_NOAA['Date Time'],errors='coerce')
pi = pi.merge(pi_NOAA,on='date',how='outer')
pi.drop(labels=range(0,2), axis = 0,inplace=True)
pi.reset_index(drop = True, inplace=True)
pi['offset'] = pi['pwl'] - pi[' Water Level']
pi = pi.drop(columns='Date Time', axis= 0)
pi = pi.drop(columns=' Sigma', axis= 0)
pi = pi.drop(columns=' O or I (for verified)', axis= 0)
pi = pi.drop(columns=' F', axis= 0)
pi = pi.drop(columns=' R', axis= 0)
pi = pi.drop(columns=' L', axis= 0)
pi = pi.drop(columns=' Quality ', axis= 0)

In [2]:
offset = pd.DataFrame()
offset['values'] = pi['offset']
offset['date'] = pi['date']
offset.replace(0, pd.NA, inplace=True)
offset = offset.dropna()
offset['change'] = (offset['values'] != offset['values'].shift()).cumsum()

result = offset.groupby('change').agg(
    start_index=('change', 'idxmin'),
    end_index=('change', 'idxmax'),
    length=('values', 'size'),
    value=('values', 'first')
).reset_index(drop=True)

# Drop the helper column if necessary
offset = offset.drop(columns=['change'])

result = result[result['length']>=10]

result.to_clipboard()

In [24]:
pi[2464340



:2464344



]

,station #,date,pwl,Water Level,offset
2464340,NaN,2018-10-25 23:24:00,NaN,2.005,NaN
2464341,NaN,2018-10-25 23:30:00,NaN,2.002,NaN
2464342,NaN,2018-10-25 23:36:00,NaN,1.997,NaN
2464343,NaN,2018-10-25 23:42:00,NaN,1.993,NaN
